# Recozimento simulado (Simulated Annealing) - Auto Featuring - Experimento

Este é um componente que usa uma técnica chamada Simulated Annealing para executar a AutoFeaturing. <br>

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
# parâmetros
dataset = "eucalyptus" #@param {type:"string", label:"Base de dados", description:"Base de dados a ser utilizada."}

target = "Utility" #@param {type:"feature", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}

date = None #@param {type:"feature", label:"Coluna de data", description:"Coluna com data que será utilizada para extrair novas características."}

group = ["Sp"] #@param {type:"feature", multiple:true, label:"Colunas para agrupar", description:"Colunas que serão utilizadas para agrupar e extrair novas características."}

# Aumentar a pesquisa de soluções
alpha = 1 #@param {type:"integer", label:"Alpha", description:"Parâmetro para aumentar ou diminuir a busca por soluções."}

## Acesso ao conjunto de dados

Utiliza a função `load_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar conjuntos de dados.
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
import pandas as pd
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]
columns = df.columns
df_features = pd.DataFrame(featuretypes)

## Chamada do Auto Featuring

In [ ]:
!wget https://raw.githubusercontent.com/platiagro/projects/master/samples/simulated-annealing/simulated.py

In [ ]:
from simulated import SimulatedAnnealing

simulated = SimulatedAnnealing(df,
                               df_features,
                               target,
                               group,
                               date,
                               alpha)

# Pre-process
simulated.preprocess()

# Autofeat
df, features, transformations = simulated.auto_feat()

## Salva alterações no conjunto de dados

Utiliza a função `save_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar alterações no conjuntos de dados.

In [ ]:
from platiagro import save_dataset

save_dataset(name=dataset, df=df)

## Salva modelo e outros artefatos

Utiliza a função `save_model` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar modelos e outros artefatos.<br>
Essa função torna estes artefatos disponíveis para o notebook de implantação.

In [ ]:
from platiagro import save_model

save_model(features=features,
           transformations=transformations,
           columns=list(columns))